In [1]:
import os;  
from ipynb.fs.full.extractfeatures import extract_features;
from ipynb.fs.full.textPreprocessor import processText;
from ipynb.fs.full.testData import load_clean_descriptions_photo_features;
from ipynb.fs.full.testData import create_tokenizer_max_length, create_sequences;
from ipynb.fs.full.model import define_model;
from keras.callbacks import ModelCheckpoint

/home/gis-local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/home/gis-local/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [2]:
# create image feature file and cleaned description of image file
if (os.path.exists("./features.pkl") ==  False):
    extract_features("../../flickr8k_dataset/Flickr8k_Dataset/Flicker8k_Dataset", "./features.pkl")
if (os.path.exists("./descriptions.txt") ==  False):
    vocabulary = processText("../../flickr8k_dataset/Flickr8k_text/Flickr8k.token.txt", "./descriptions.txt")

In [3]:
# prepare train dataset with startseq desc endseq.
train_descriptions, train_features  = load_clean_descriptions_photo_features("./descriptions.txt", "/home/gis-local/Desktop/built.io/deepakWork/flickr8k_dataset/Flickr8k_text/Flickr_8k.trainImages.txt", "./features.pkl")
# prepare tokenizer and determine max length of desc
tokenizer, max_length, vocab_size = create_tokenizer_max_length(train_descriptions)

Descriptions: train=6000
Photos: train=6000
Vocabulary Size: 7579
Description Length: 34


In [4]:
# prepare sequences
# X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)

In [5]:
# X1train.shape, X2train.shape, ytrain.shape, ytrain

In [6]:
# prepare test dataset with startseq desc endseq.
# test_descriptions, test_features  = load_clean_descriptions_photo_features("./descriptions.txt", "/home/gis-local/Desktop/built.io/deepakWork/flickr8k_dataset/Flickr8k_text/Flickr_8k.testImages.txt", "./features.pkl")
# prepare sequences
# X1test, X2test, ytest, vocab_size = create_sequences(tokenizer, max_length, test_descriptions, test_features, vocab_size)


In [7]:
# define the model
model = define_model(vocab_size, max_length)
# define checkpoint callback
# filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# # fit model
# model.fit([X1train, X2train], ytrain, epochs=1, verbose=2, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 256)      1940224     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           input_2[0][0]                    
__________________________________________________________________________________________________
dropout_2 

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length):
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            # retrieve the photo feature
            photo = photos[key][0]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
            yield [[in_img, in_seq], out_word]
            
# run in progressive loading if you don't have enough memory
# train the model, run epochs manually and save after each epoch
epochs = 20
steps = len(train_descriptions)
for i in range(epochs):
    print("runnning epochs:", i)
    # create the data generator
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    # fit for one epoch
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    # save model
    model.save('model_' + str(i) + '.h5')

runnning epochs: 0
Epoch 1/1
6000/6000 [==============================] - 1235s 206ms/step - loss: 4.6793
runnning epochs: 1
Epoch 1/1
6000/6000 [==============================] - 1302s 217ms/step - loss: 3.9168
runnning epochs: 2
Epoch 1/1
6000/6000 [==============================] - 1273s 212ms/step - loss: 3.6553
runnning epochs: 3
Epoch 1/1
6000/6000 [==============================] - 1307s 218ms/step - loss: 3.5099
runnning epochs: 4
Epoch 1/1
6000/6000 [==============================] - 1405s 234ms/step - loss: 3.4145
runnning epochs: 5
Epoch 1/1
6000/6000 [==============================] - 1397s 233ms/step - loss: 3.3445
runnning epochs: 6
Epoch 1/1
6000/6000 [==============================] - 1548s 258ms/step - loss: 3.2960
runnning epochs: 7
Epoch 1/1
5330/6000 [=========================>....] - ETA: 3:24 - loss: 3.2621